## 02 - INITIAL DATA ANALYSIS

###### Lecture explaining xarray functionalities geospatial processing : datacubes_data_handling.ipynb

#### 01.1 - Libraries import

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr

In [38]:
# Open the file NetCDF
dataset1 = xr.open_dataset(r"..\..\data\data_downloads\era5-downscaled-over-italy_hourly_437292.nc")
dataset2 = xr.open_dataset(r"..\..\data\data_downloads\air_temperature_era5-downscaled-over-italy_hourly_442373.nc")

In [39]:
def calculate_relative_humidity(Ta, Td):
    RH = 100 * np.exp((17.625 * Td) / (243.04 + Td) - (17.625 * Ta) / (243.04 + Ta))
    return RH

def calculate_heat_index(Ta, RH):
    HI = (-42.379 + 2.04901523 * Ta + 10.14333127 * RH
          - 0.22475541 * Ta * RH - 6.8378e-3 * Ta**2
          - 5.48172e-2 * RH**2 + 1.229e-3 * Ta**2 * RH
          + 8.528e-4 * Ta * RH**2 - 1.99e-6 * Ta**2 * RH**2)
    return HI

def calculate_humidex(Ta_K, Td_K):    
    # Evita valori troppo bassi per Td (dew point)
    Td_K = np.maximum(Td_K, 200)  
    
    # Calcolo dell'Humidex con formula Masterson
    exponent = np.clip(5417.7530 * ((1/273.16) - (1/Td_K)), a_min=None, a_max=500000)
    Hu_K = Ta_K + 0.5555 * (6.11 * np.exp(exponent) - 10)
    
    # Convertiamo alla fine da Kelvin a Celsius
    return Hu_K - 273.15

# Wet Bulb Globe Temperature
def calculate_wbt(Ta, RH):       
    WBT = (Ta * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) +
           np.arctan(Ta + RH) - np.arctan(RH - 1.676331) +
           0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - 4.686035)
    return WBT

# Wet Bulb Temperature
def calculate_wbgt(Ta, WBT):
    WBGT = 0.7 * WBT + 0.3 * Ta
    return WBGT

def calculate_lethal_heat_stress_index(WBT, RH):
    Ls = WBT + 4.5 * (1 - (RH / 100)**2)
    return Ls

def calculate_utci(Ta, RH):
    pa = RH / 100 * 6.105 * np.exp(17.27 * Ta / (237.7 + Ta))
    UTCI = Ta + 0.607562 + 0.022771 * Ta - 0.003578 * RH - 0.000119 * Ta * RH
    return UTCI

#### Verifica sul singolo timestamp del superamento o meno di tutte le soglie di pericolosità

In [87]:
import numpy as np
import pandas as pd
import xarray as xr

# **Seleziona il primo timestamp disponibile**
time_selected = dataset2['T_2M'].time[23475]

# **Estrai temperatura e dew point temperature per il primo timestamp**
temperature_snapshot = dataset2['T_2M'].sel(time=time_selected)
dew_point_snapshot = dataset1['TD_2M'].sel(time=time_selected)

# **Filtra valori anomali e interpoliamo dew point temperature**
dew_point_filtered = dew_point_snapshot.where(dew_point_snapshot > 243.15)
dew_point_interpolated = dew_point_filtered.interpolate_na(dim='rlat', method='linear').interpolate_na(dim='rlon', method='linear')

# **Calcola gli indici per ciascun pixel**
humidex_snapshot = calculate_humidex(temperature_snapshot, dew_point_interpolated)
relative_humidity_snapshot = calculate_relative_humidity(temperature_snapshot - 273.15, dew_point_interpolated - 273.15)
heat_index_snapshot = calculate_heat_index(temperature_snapshot - 273.15, relative_humidity_snapshot)
wbt_snapshot = calculate_wbt(temperature_snapshot - 273.15, relative_humidity_snapshot)
wbgt_snapshot = calculate_wbgt(temperature_snapshot - 273.15, wbt_snapshot)
lhs_snapshot = calculate_lethal_heat_stress_index(wbt_snapshot, relative_humidity_snapshot)
utci_snapshot = calculate_utci(temperature_snapshot - 273.15, relative_humidity_snapshot)

# **Calcola statistiche per ciascun indice**
def calculate_stats(data_array):
    return {
        "mean": data_array.mean().item(),
        "median": data_array.median().item(),
        "p95": np.percentile(data_array, 95),
        "p99": np.percentile(data_array, 99),
        "max": data_array.max().item()
    }

indices = {
    "Heat Index": heat_index_snapshot,
    "Humidex": humidex_snapshot,
    "Lethal Heat Stress Index": lhs_snapshot,
    "UTCI": utci_snapshot,
    "WBGT": wbgt_snapshot
}

stats_results = {key: calculate_stats(value) for key, value in indices.items()}

# **Definizione delle soglie per ciascun indice**
soglie = {
    "Heat Index": 40.6,
    "Humidex": 45,
    "Lethal Heat Stress Index": 27,
    "UTCI": 44,      # By literature is 46
    "WBGT": 28       # By literature is 30
}

# **Creazione del DataFrame con i risultati**
df_results = pd.DataFrame([
    {
        "Indice": key,
        "Media (°C)": value["mean"],
        "Mediana (°C)": value["median"],
        "95° Perc. (°C)": value["p95"],
        "99° Perc. (°C)": value["p99"],
        "Massimo (°C)": value["max"],
        "Soglia (°C)": soglie[key],
        "Media > Soglia": "si" if value["mean"] > soglie[key] else "no",
        "Mediana > Soglia": "si" if value["median"] > soglie[key] else "no",
        "95° Perc. > Soglia": "si" if value["p95"] > soglie[key] else "no",
        "99° Perc. > Soglia": "si" if value["p99"] > soglie[key] else "no",
        "Massimo > Soglia": "si" if value["max"] > soglie[key] else "no"
    }
    for key, value in stats_results.items()
])

# **Ordina il DataFrame**
df_results.sort_values(by="Indice", inplace=True)

# **Funzione per colorare i "si" di rosso e i "no" di verde**
def color_superamento(val):
    color = "red" if val == "si" else "green"
    return f"color: {color}"

# **Applica la formattazione condizionale**
df_styled = df_results.style.applymap(color_superamento, subset=["Media > Soglia", "Mediana > Soglia", "95° Perc. > Soglia", "99° Perc. > Soglia", "Massimo > Soglia"])

# **Visualizza il DataFrame formattato con il timestamp in analisi**
print(f"\n=== Confronto Indici vs Soglie per il timestamp selezionato ({str(time_selected.values)}) ===")
df_styled


=== Confronto Indici vs Soglie per il timestamp selezionato (2023-07-19T14:00:00.000000000) ===


,Indice,Media (°C),Mediana (°C),95° Perc. (°C),99° Perc. (°C),Massimo (°C),Soglia (°C),Media > Soglia,Mediana > Soglia,95° Perc. > Soglia,99° Perc. > Soglia,Massimo > Soglia
0,Heat Index,111.683800,91.658463,233.415842,277.036213,341.995575,40.600000,si,si,si,si,si
1,Humidex,36.104279,40.435208,45.722592,46.640403,47.901360,45.000000,no,no,si,si,si
2,Lethal Heat Stress Index,23.854443,25.762875,28.292885,28.847029,29.360319,27.000000,no,no,si,si,si
3,UTCI,35.516464,38.994286,44.836285,45.557074,46.906635,44.000000,no,no,si,si,si
4,WBGT,24.126650,26.631012,29.446013,29.923445,30.508659,28.000000,no,no,si,si,si


| Criteri per Identificare le Ondate di Calore | Valori soglia che definiscono una condizione di ondata di calore pericolosa |
|:--|:--|
| Heat Index (HI) | > 105°F (~40.6°C) |
| Humidex (Hu) | > 45°C |
| WBGT | > 30°C |
| Lethal Heat Stress Index (Ls) | > 27°C |
| UTCI | > 46°C |

#### Verifica su tutti i timestamp di un anno del superamento o meno di tutte le soglie di pericolosità & selezione dei giorni consecutivi in cui ciò accade -> heat wave vs casi isolati

In [100]:
import numpy as np
import pandas as pd
import xarray as xr

# **Seleziona un anno specifico per l'analisi**
year = 2022
timestamps = dataset2['T_2M'].time.sel(time=dataset2['T_2M'].time.dt.year == year)

# **Definizione delle soglie per ciascun indice**
soglie = {
    "Heat Index": 40.6,
    "Humidex": 45,
    "Lethal Heat Stress Index": 27,
    "UTCI": 46,
    "WBGT": 30
}

# **Lista per salvare i risultati finali**
heatwave_records = []

# **Itera su tutti i timestamp dell'anno selezionato**
for i, time_selected in enumerate(timestamps):
    
    # **Estrai temperatura e dew point per il timestamp attuale**
    temperature_snapshot = dataset2['T_2M'].sel(time=time_selected)
    dew_point_snapshot = dataset1['TD_2M'].sel(time=time_selected)
    
    # **Filtra valori anomali e interpoliamo dew point temperature**
    dew_point_filtered = dew_point_snapshot.where(dew_point_snapshot > 243.15)
    dew_point_interpolated = dew_point_filtered.interpolate_na(dim='rlat', method='linear').interpolate_na(dim='rlon', method='linear')

    # **Calcola gli indici per ciascun pixel**
    humidex_snapshot = calculate_humidex(temperature_snapshot, dew_point_interpolated)
    relative_humidity_snapshot = calculate_relative_humidity(temperature_snapshot - 273.15, dew_point_interpolated - 273.15)
    heat_index_snapshot = calculate_heat_index(temperature_snapshot - 273.15, relative_humidity_snapshot)
    wbt_snapshot = calculate_wbt(temperature_snapshot - 273.15, relative_humidity_snapshot)
    wbgt_snapshot = calculate_wbgt(temperature_snapshot - 273.15, wbt_snapshot)
    lhs_snapshot = calculate_lethal_heat_stress_index(wbt_snapshot, relative_humidity_snapshot)
    utci_snapshot = calculate_utci(temperature_snapshot - 273.15, relative_humidity_snapshot)

    # **Calcola statistiche per ciascun indice**
    def calculate_stats(data_array):
        return {
            "mean": data_array.mean().item(),
            "median": data_array.median().item(),
            "p95": np.percentile(data_array, 95),
            "p99": np.percentile(data_array, 99),
            "max": data_array.max().item()
        }

    indices = {
        "Heat Index": heat_index_snapshot,
        "Humidex": humidex_snapshot,
        "Lethal Heat Stress Index": lhs_snapshot,
        "UTCI": utci_snapshot,
        "WBGT": wbgt_snapshot
    }

    stats_results = {key: calculate_stats(value) for key, value in indices.items()}

    # **Verifica se TUTTI gli indici superano almeno in una delle colonne le soglie**
    all_exceed = all(
        any([
            stats_results[key]["mean"] > soglie[key],
            stats_results[key]["median"] > soglie[key],
            stats_results[key]["p95"] > soglie[key],
            stats_results[key]["p99"] > soglie[key],
            stats_results[key]["max"] > soglie[key]
        ])
        for key in indices.keys()
    )

    # **Se la condizione è soddisfatta, salva il risultato**
    if all_exceed:
        
        df_results = pd.DataFrame([
            {
                "Indice": key,
                "Media (°C)": value["mean"],
                "Mediana (°C)": value["median"],
                "95° Perc. (°C)": value["p95"],
                "99° Perc. (°C)": value["p99"],
                "Massimo (°C)": value["max"],
                "Soglia (°C)": soglie[key],
                "Media > Soglia": "si" if value["mean"] > soglie[key] else "no",
                "Mediana > Soglia": "si" if value["median"] > soglie[key] else "no",
                "95° Perc. > Soglia": "si" if value["p95"] > soglie[key] else "no",
                "99° Perc. > Soglia": "si" if value["p99"] > soglie[key] else "no",
                "Massimo > Soglia": "si" if value["max"] > soglie[key] else "no"
            }
            for key, value in stats_results.items()
        ])
        
        # **Aggiunta di timestamp nel dataframe per chiarezza**
        df_results.insert(0, "Timestamp", str(time_selected.values))
        df_results.insert(1, "Numero Timestamp", i)

        # **Aggiungi i risultati alla lista per il salvataggio**
        heatwave_records.extend(df_results.to_dict(orient="records"))

# **Creazione del DataFrame finale**
df_heatwave = pd.DataFrame(heatwave_records)

# **Identificazione di heat wave e casi isolati**
df_heatwave["Data"] = pd.to_datetime(df_heatwave["Timestamp"]).dt.date
df_heatwave["Heat Wave"] = "caso isolato"

for i, row in df_heatwave.iterrows():
    current_date = row["Data"]
    next_day = current_date + pd.Timedelta(days=1)
    prev_day = current_date - pd.Timedelta(days=1)

    if ((next_day in df_heatwave["Data"].values) or (prev_day in df_heatwave["Data"].values)):
        df_heatwave.at[i, "Heat Wave"] = "heat wave"

# **Salvataggio su file CSV con nome contenente l'anno**
df_heatwave.to_csv(f"heatwave_timestamps_{year}.csv", index=False)

# **Controllo per stampare il messaggio appropriato**
print("\n=== Giorni con Ondate di Calore Estreme dell'anno selezionato ===")
if df_heatwave.empty:
    print("Nessun giorno dell'anno selezionato ha superato tutte le soglie di pericolosità per il caldo estremo nel 95° percentile.")
else:
    print("Stampa nella cella successiva df_heatwave per vederlo in formato data frame")


=== Giorni con Ondate di Calore Estreme dell'anno selezionato ===
Stampa nella cella successiva df_heatwave per vederlo in formato data frame


In [101]:
df_heatwave

,Timestamp,Numero Timestamp,Indice,Media (°C),Mediana (°C),95° Perc. (°C),99° Perc. (°C),Massimo (°C),Soglia (°C),Media > Soglia,Mediana > Soglia,95° Perc. > Soglia,99° Perc. > Soglia,Massimo > Soglia,Data,Heat Wave
0,2022-06-27T14:00:00.000000000,159,Heat Index,110.022453,90.614151,200.401920,252.054794,337.978729,40.6,si,si,si,si,si,2022-06-27,caso isolato
1,2022-06-27T14:00:00.000000000,159,Humidex,35.428917,39.234235,44.950700,45.796616,47.226297,45.0,no,no,no,si,si,2022-06-27,caso isolato
2,2022-06-27T14:00:00.000000000,159,Lethal Heat Stress Index,23.687557,25.612419,27.823743,28.416399,29.362671,27.0,no,no,si,si,si,2022-06-27,caso isolato
3,2022-06-27T14:00:00.000000000,159,UTCI,34.888977,38.254417,43.726063,44.745889,46.285069,46.0,no,no,no,no,si,2022-06-27,caso isolato
4,2022-06-27T14:00:00.000000000,159,WBGT,23.791891,26.050282,29.040370,29.462555,30.303282,30.0,no,no,no,no,si,2022-06-27,caso isolato
5,2022-07-22T14:00:00.000000000,309,Heat Index,82.583443,71.013199,134.669099,208.029107,297.734039,40.6,si,si,si,si,si,2022-07-22,heat wave
6,2022-07-22T14:00:00.000000000,309,Humidex,36.424200,39.860176,45.130687,45.985631,47.003825,45.0,no,no,si,si,si,2022-07-22,heat wave
7,2022-07-22T14:00:00.000000000,309,Lethal Heat Stress Index,23.211386,24.606380,27.437568,28.053050,28.470070,27.0,no,no,si,si,si,2022-07-22,heat wave
8,2022-07-22T14:00:00.000000000,309,UTCI,37.570866,40.906570,45.116886,45.808594,46.608528,46.0,no,no,no,no,si,2022-07-22,heat wave
9,2022-07-22T14:00:00.000000000,309,WBGT,24.091400,26.006790,29.009311,29.470568,30.038940,30.0,no,no,no,no,si,2022-07-22,heat wave


#### GIF rappresentante il periodo prima durante e dopo la heat wave

In [123]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import imageio.v2 as imageio  # Evita il warning di ImageIO
import os

# **Parametri personalizzabili**
year = 2022  # Anno di analisi
target_timestamp_index = 309  # Numero del timestamp di riferimento (cambiabile dall'utente)
days_before = 10  # Giorni precedenti da mostrare
days_after = 12  # Giorni successivi da mostrare
indice_selezionato = "Heat Index"  # Indice da visualizzare nella mappa

# **Caricamento dei dati**
timestamps = dataset2['T_2M'].time.sel(time=dataset2['T_2M'].time.dt.year == year)
df_heatwave["Data"] = pd.to_datetime(df_heatwave["Timestamp"]).dt.date
heatwave_days = df_heatwave[df_heatwave['Heat Wave'] == 'heat wave']['Data'].unique()
heatwave_dates = set(str(date) for date in heatwave_days)  # Set di date formattate

# **Seleziona il timestamp di riferimento**
target_time = timestamps[target_timestamp_index]

# **Definiamo l'intervallo temporale per la GIF**
start_time = target_time - np.timedelta64(days_before, 'D')
end_time = target_time + np.timedelta64(days_after, 'D')
time_range = timestamps.sel(time=(timestamps >= start_time) & (timestamps <= end_time))

# **Calcola il range fisso per la legenda solo sui timestamp selezionati**
heat_index_values = []
for time_selected in time_range:
    temperature_snapshot = dataset2['T_2M'].sel(time=time_selected)
    dew_point_snapshot = dataset1['TD_2M'].sel(time=time_selected)
    dew_point_filtered = dew_point_snapshot.where(dew_point_snapshot > 243.15)
    dew_point_interpolated = dew_point_filtered.interpolate_na(dim='rlat', method='linear').interpolate_na(dim='rlon', method='linear')
    heat_index_snapshot = calculate_heat_index(temperature_snapshot - 273.15, calculate_relative_humidity(temperature_snapshot - 273.15, dew_point_interpolated - 273.15))
    heat_index_values.append(heat_index_snapshot.values)

heat_index_values = np.concatenate([hi.flatten() for hi in heat_index_values])
heat_index_min = heat_index_values.min()
heat_index_max = heat_index_values.max()

# **Creazione di una cartella temporanea per le immagini**
output_folder = "temp_frames"
os.makedirs(output_folder, exist_ok=True)

# **Genera i frame per la GIF**
frames = []
for time_selected in time_range:
    
    # **Estrai i dati di temperatura e dew point**
    temperature_snapshot = dataset2['T_2M'].sel(time=time_selected)
    dew_point_snapshot = dataset1['TD_2M'].sel(time=time_selected)
    
    # **Filtra valori anomali e interpoliamo dew point temperature**
    dew_point_filtered = dew_point_snapshot.where(dew_point_snapshot > 243.15)
    dew_point_interpolated = dew_point_filtered.interpolate_na(dim='rlat', method='linear').interpolate_na(dim='rlon', method='linear')
    
    # **Calcola gli indici**
    heat_index_snapshot = calculate_heat_index(temperature_snapshot - 273.15, calculate_relative_humidity(temperature_snapshot - 273.15, dew_point_interpolated - 273.15))
    
    # **Plotta la distribuzione dell'Heat Index per il timestamp corrente**
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(heat_index_snapshot, cmap="inferno", origin="lower", vmin=heat_index_min, vmax=heat_index_max)
    
    # **Legenda fissa**
    cbar = plt.colorbar(im, label="Heat Index (°C)")
    cbar.ax.tick_params(labelsize=10)
    
    # **Aggiungi il titolo con il timestamp e indicazione della heatwave**
    timestamp_str = str(time_selected.values)[:16]
    ax.set_title(timestamp_str, fontsize=14, fontweight="bold", ha='left', x=0)
    
    # **Se il giorno è una heatwave, evidenzia il riquadro e mostra la scritta**
    date_str = timestamp_str[:10]  # Estrai solo la parte di data
    if date_str in heatwave_dates:
        for spine in ax.spines.values():
            spine.set_edgecolor('red')
            spine.set_linewidth(3)
        ax.text(0.5, -0.1, "HEAT WAVE HIT", fontsize=18, color='red', ha='center', transform=ax.transAxes, fontweight='bold')
    
    # **Sostituisci i caratteri speciali per evitare errori nei nomi dei file**
    safe_timestamp = timestamp_str.replace(":", "-").replace("T", "_").split(".")[0]
    frame_filename = os.path.join(output_folder, f"frame_{safe_timestamp}.png")

    # **Salva il frame temporaneo**
    plt.savefig(frame_filename, dpi=100)
    plt.close()
    
    frames.append(frame_filename)

# **Creazione della GIF**
gif_filename = f"heatwave_evolution_{year}.gif"
with imageio.get_writer(gif_filename, mode='I', duration=5) as writer:
    for frame in frames:
        image = imageio.imread(frame)
        writer.append_data(image)

# **Pulizia della cartella temporanea**
for frame in frames:
    try:
        os.remove(frame)
    except PermissionError:
        pass  # Ignora errori se il file è ancora in uso
try:
    os.rmdir(output_folder)
except PermissionError:
    pass  # Ignora errori se la cartella è ancora in uso

print(f"\n GIF creata con successo: {gif_filename}")


 GIF creata con successo: heatwave_evolution_2022.gif


#### Iterazione dall'inizio del dataset del superamento o meno di tutte le soglie di pericolosità per tutti gli anni, che si arresta quando trovato un anno con almeno un timestamp che supera tutte le soglie di pericolosità

In [19]:
# **Definizione delle soglie per ciascun indice**
soglie = {
    "Heat Index": 40.6,
    "Humidex": 45,
    "Lethal Heat Stress Index": 27,
    "UTCI": 46,
    "WBGT": 30
}

# **Ottieni tutti gli anni disponibili nel dataset**
anni_disponibili = np.unique(dataset2['T_2M'].time.dt.year.values)

# **Variabile per verificare se abbiamo trovato un anno con heatwave**
anno_trovato = False

# **Iteriamo su tutti gli anni disponibili**
for year in anni_disponibili:
    if anno_trovato:
        break  # Se abbiamo già trovato un anno valido, interrompiamo il loop

    print(f"\n🔍 Analizzando l'anno {year}...")

    # **Seleziona i timestamp per l'anno corrente**
    timestamps = dataset2['T_2M'].time.sel(time=dataset2['T_2M'].time.dt.year == year)

    # **Lista per salvare i giorni in cui tutte le soglie vengono superate**
    heatwave_days = []

    # **Itera su tutti i timestamp dell'anno selezionato**
    for time_selected in timestamps:
        
        # **Estrai temperatura e dew point per il timestamp attuale**
        temperature_snapshot = dataset2['T_2M'].sel(time=time_selected)
        dew_point_snapshot = dataset1['TD_2M'].sel(time=time_selected)
        
        # **Filtra valori anomali e interpoliamo dew point temperature**
        dew_point_filtered = dew_point_snapshot.where(dew_point_snapshot > 243.15)
        dew_point_interpolated = dew_point_filtered.interpolate_na(dim='rlat', method='linear').interpolate_na(dim='rlon', method='linear')

        # **Calcola gli indici per ciascun pixel**
        humidex_snapshot = calculate_humidex(temperature_snapshot, dew_point_interpolated)
        relative_humidity_snapshot = calculate_relative_humidity(temperature_snapshot - 273.15, dew_point_interpolated - 273.15)
        heat_index_snapshot = calculate_heat_index(temperature_snapshot - 273.15, relative_humidity_snapshot)
        wbt_snapshot = calculate_wbt(temperature_snapshot - 273.15, relative_humidity_snapshot)
        wbgt_snapshot = calculate_wbgt(temperature_snapshot - 273.15, wbt_snapshot)
        lhs_snapshot = calculate_lethal_heat_stress_index(wbt_snapshot, relative_humidity_snapshot)
        utci_snapshot = calculate_utci(temperature_snapshot - 273.15, relative_humidity_snapshot)

        # **Calcola media e mediana per ciascun indice**
        humidex_mean, humidex_median = humidex_snapshot.mean().item(), humidex_snapshot.median().item()
        heat_index_mean, heat_index_median = heat_index_snapshot.mean().item(), heat_index_snapshot.median().item()
        wbgt_mean, wbgt_median = wbgt_snapshot.mean().item(), wbgt_snapshot.median().item()
        lhs_mean, lhs_median = lhs_snapshot.mean().item(), lhs_snapshot.median().item()
        utci_mean, utci_median = utci_snapshot.mean().item(), utci_snapshot.median().item()

        # **Verifica se tutti gli indici superano le soglie contemporaneamente**
        all_indices_above_threshold = (
            (heat_index_mean > soglie["Heat Index"]) and (heat_index_median > soglie["Heat Index"]) and
            (humidex_mean > soglie["Humidex"]) and (humidex_median > soglie["Humidex"]) and
            (lhs_mean > soglie["Lethal Heat Stress Index"]) and (lhs_median > soglie["Lethal Heat Stress Index"]) and
            (utci_mean > soglie["UTCI"]) and (utci_median > soglie["UTCI"]) and
            (wbgt_mean > soglie["WBGT"]) and (wbgt_median > soglie["WBGT"])
        )

        # **Se la condizione è soddisfatta, aggiungiamo il giorno alla lista**
        if all_indices_above_threshold:
            heatwave_days.append(str(time_selected.values))
            anno_trovato = True  # Imposta il flag per fermare la ricerca dopo questo anno

    # **Se abbiamo trovato almeno un giorno, interrompiamo il loop**
    if anno_trovato:
        break

# **Se abbiamo trovato un anno valido, creiamo il DataFrame**
if anno_trovato and heatwave_days:
    df_heatwave_days = pd.DataFrame({"Heatwave Days": heatwave_days})

    # **Ordina i risultati in base alla data**
    df_heatwave_days.sort_values(by="Heatwave Days", inplace=True)

    # **Esporta il DataFrame in un file CSV per visualizzazione**
    df_heatwave_days.to_csv(f"heatwave_days_{year}.csv", index=False)

    # **Stampa i risultati**
    print(f"\n=== Giorni con Ondate di Calore Estreme nell'anno {year} ===")
    print(df_heatwave_days.to_string(index=False))

else:
    print("\n Nessun anno nel dataset ha superato tutte le soglie di pericolosità per il caldo estremo")



🔍 Analizzando l'anno 1981...

🔍 Analizzando l'anno 1982...

🔍 Analizzando l'anno 1983...

🔍 Analizzando l'anno 1984...

🔍 Analizzando l'anno 1985...

🔍 Analizzando l'anno 1986...

🔍 Analizzando l'anno 1987...

🔍 Analizzando l'anno 1988...

🔍 Analizzando l'anno 1989...

🔍 Analizzando l'anno 1990...

🔍 Analizzando l'anno 1991...

🔍 Analizzando l'anno 1992...

🔍 Analizzando l'anno 1993...

🔍 Analizzando l'anno 1994...

🔍 Analizzando l'anno 1995...

🔍 Analizzando l'anno 1996...

🔍 Analizzando l'anno 1997...

🔍 Analizzando l'anno 1998...

🔍 Analizzando l'anno 1999...

🔍 Analizzando l'anno 2000...

🔍 Analizzando l'anno 2001...

🔍 Analizzando l'anno 2002...

🔍 Analizzando l'anno 2003...

🔍 Analizzando l'anno 2004...

🔍 Analizzando l'anno 2005...

🔍 Analizzando l'anno 2006...

🔍 Analizzando l'anno 2007...

🔍 Analizzando l'anno 2008...

🔍 Analizzando l'anno 2009...

🔍 Analizzando l'anno 2010...

🔍 Analizzando l'anno 2011...

🔍 Analizzando l'anno 2012...

🔍 Analizzando l'anno 2013...

🔍 Analizz

KeyboardInterrupt: 